In [ ]:
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim import Adam

#-------------

import poseigen_seaside.basics as se
import poseigen_seaside.metrics as mex

import poseigen_trident.utils as tu
import poseigen_trident.preassembled as pa
import poseigen_compass as co

In [ ]:
datafolder = se.NewFolder('data')
xpsfolder = se.NewFolder('xps')

#--------------------------------------

synth_repeats = 5

num_obs = [5000, 10000, 20000] #########
num_obs_ids = ['O' + str(1+x) for x in np.arange(len(num_obs))]

noise_lvls = [0, 0.5, 1.0] #################
noise_lvls_ids = ['N' + str(1+x) for x in np.arange(len(noise_lvls))]

#--------------------------------------

snxs = se.PickleLoad('snxs')
snx2 = snxs[1]

In [5]:
synth_data = se.PickleLoad(datafolder + 'synth_data')
synth_splits = se.PickleLoad(datafolder + 'synth_splits')

In [ ]:
TCR_repeats = 15

TT_base = {'loss_bind': False, 'smallest': True, 'batchsize': 512,
            'opt': Adam, 'maxepochs': 200, 'patience': 10, 'pickup': True, 
            'duds': TCR_repeats, 'duds_mode': [tu.SimpNetReset, {}], 
            'statusprints': 10}

TCS_base = {'trainer': tu.TridentTrainer, 'smallest': None,
            'get_predictions': True, 'pred_rewrite': False, 
            'add_pred_args': {'batchsize': 512}, 'score_on': 2}


TCR_base = {'Splits': None, 'repeats': TCR_repeats, 
            'pickup': False, 'savemodels': True, 'returnmodel': True}

In [7]:
def Adder(inp, add, scalefactor = 1):
    return (inp * scalefactor) + add
def AntiLog(inp, added, add = 0, scalefactor = 1): 
    return (math.e ** (inp * scalefactor)) - added + add


def XpsBootstrapEnsemb(combs = None, 
                       pathname = None, pathname_ext = None,
                       repeats = None, metrics_mode = None, 
                       
                       pred_scalefactors = None,

                       loggos = False,

                       load_bootstrap_preds = None,
                       
                       out = None, out_std = None, out_weights = None, out_bind = None,
                       split = None, score_on = 1,

                       iterx = 100, rewrite = False): 
    
    #load_bootstrap_preds IS THE EXTENSION OF THE COMPARATOR!!!!!!!!!!!!!!!!!!!!!
    
    ps_args = {'out': out, 'out_std': out_std, 
               'out_weights': out_weights, 'out_bind': out_bind,
               'split': split, 'metrics_mode': metrics_mode}
    
    es_args_base = { 'score_on': score_on,
                    'std_cutoff': None, 'ddof': 1, 
                    'top': 3, 'smallest': True}
    
    #################################

    boots_scores, boots_idxs = [], []

    if pathname_ext is None: pathname_ext = ''
    else: pathname_ext = '_' + pathname_ext

    if isinstance(loggos, list) is False: loggos = [loggos] * len(combs)
    elif len(loggos) == 1: loggos = loggos * len(combs)

    for icom, com in combs: 

        #################################

        lo = loggos[icom]

        print(lo)

        qw = 'log' if lo else 'nolog'
        sf = pred_scalefactors[qw]

        if lo: pm_mode = [AntiLog, {'added': 1, 'add': 0, 'scalefactor': sf}]
        else: pm_mode = [Adder, {'add': 0, 'scalefactor': sf}]

        ps_args_com = {**ps_args, 'pred_modif_mode': pm_mode}
    


        ######################################

        pn_t = se.NewFolder(pathname + str(icom))

        pn_preds, pn_mods = [[pn_t + '0_' + str(ir) + s for ir in range(repeats)] 
                            for s in ['_preds.p',  '_Mod.pt']]
        
        #################################

        pn_e = pn_t + 'boots_ensemb' + pathname_ext

        pn_s = se.NewFolder(pn_t + 'EnsembStuff' + pathname_ext)
        
        if os.path.isfile(pn_e + '.p') and rewrite is False: 

            boots_ensemb = se.PickleLoad(pn_e)
            #print('already did it')

        else: 

            if load_bootstrap_preds is None: 

                es_args_com = {**ps_args_com, **es_args_base}
            
                boots_ensemb = co.Bootstrapper(pn_preds, 
                                            mode = [co.EnsembleScorer, es_args_com],
                                            pathname = pn_s,
                                            iters = iterx, updates = 10, return_idx = True)
            
            else: 
                boots_ensemb = []

                
                for ix in np.arange(iterx): 

                    pn_s2 = se.NewFolder(pn_t + 'EnsembStuff' + '_' + load_bootstrap_preds)

                    pn_pk = se.NewFolder(pn_s2 + str(ix)) + 'ensemble_preds_keep'
                    preds_keep = se.PickleLoad(pn_pk)
                    preds_ensemb = np.mean(preds_keep, axis = 0)
                    preds_ensemb_scores = co.PredScorer(preds_ensemb, **ps_args_com)
                    boots_ensemb.append(preds_ensemb_scores)

                boots_ensemb = [boots_ensemb, ['used load_bootstrap_preds']]
            
            se.PickleDump(boots_ensemb, pn_e)

        boots_scores.append(boots_ensemb[0])
        boots_idxs.append(boots_ensemb[1])

    
    return np.array(boots_scores), np.array(boots_idxs)


In [8]:
iterx = 50 ##########

rewrite_ensemb = False ##############

ref_ic = 0

pf_args = {'mode1': [se.RelativeChange, {'perc': True}]}

mmox = [tu.DeviaError, {'scalefactor': 1, 'expo': 2, 'root': True, 
                        'pseudo': 1e-10, 'pyt': False}]

xbe_args_base = {'repeats': TCR_repeats, 'metrics_mode': mmox,
                 'out_weights': None, 'out_bind': None,
                 'score_on': 1, 'iterx': iterx}

onesided = None
conf_alpha = 0.95

# xpLR: Learning Rate

In [ ]:
RMSE_mode = [mex.AError, {'expo': 2, 'root', True}]           

In [13]:
synth_repeats = 1

In [14]:
pn_xpLR = se.NewFolder(xpsfolder + 'xpLR' + str(0))  #################################
#T2 corrects for adding pseudo value in deviation loss instead of in the values. 

xpLR_variables = ['Learning Rate']

learners = np.round(se.GeomNumSpacing(0.0001, 0.1, 10, plier = 2), 4)[::2]
# print(learners)

xpLR_combs = [[ic, [c]] for ic, c in enumerate(learners)]
xpLR_combs

[[0, [np.float64(0.0001)]],
 [1, [np.float64(0.0007)]],
 [2, [np.float64(0.003)]],
 [3, [np.float64(0.0124)]],
 [4, [np.float64(0.05)]]]

In [ ]:
gd = 'd_mode'

for syr in np.arange(synth_repeats):
   for inobs, nobs in enumerate(num_obs):
     for inx, nx in enumerate(noise_lvls): 
        for icom, com in xpLR_combs:

            pn_t1 = se.NewFolder(pn_xpLR + str(syr))
            pn_t2 = se.NewFolder(pn_t1 + str(inobs))
            pn_t3 = se.NewFolder(pn_t2 + str(inx))
            pn_t4 = se.NewFolder(pn_t3 + str(icom))

            lrz = com[0]

            #-------------------------------------

            DX, DY = [synth_data[syr][inobs][inx][ss] for ss in ['d_x', gd]]
   

            #-------------------------------------

            DZ = None

            lmo = RMSE_mode
            mmo = lmo                        

            actf = tu.PseudoReLU(pseudo = 0)

            #-------------------------------------

            xpLR_TT_args = {'inps': [DX], 'out': DY, 
                        'out_std': DZ, 'out_bind': None,
                        'Split': synth_splits[syr][inobs][inx],
                        'loss_mode': lmo, 'metrics_mode': mmo, 
                        **TT_base}
            
            xpLR_TCS_args = {'trainer_args': xpLR_TT_args, **TCS_base}

            xpLR_TCR_args = {**xpLR_TCS_args, **TCR_base, 'pathname': pn_t4}

            #-------------------------------------

            bo = None

            mod_args = {'learningrate': lrz, 'batchsize': 512, 
                        'activation_f': actf, 'two_outs': False, 
                        'comb': bo, **snx2}

            #-------------------------------------

            _, _= tu.TridentCanRepeater(pa.SimpNet, mod_args, data = None, **xpLR_TCR_args) 

 Cross val 1 of 1, Repeat 1 of 15
0.08923178282579565
 Cross val 1 of 1, Repeat 2 of 15
0.09998472622265875
 Cross val 1 of 1, Repeat 3 of 15
0.08594804668893229
 Cross val 1 of 1, Repeat 4 of 15
0.09612217247157394
 Cross val 1 of 1, Repeat 5 of 15
0.07939346495868882
 Cross val 1 of 1, Repeat 6 of 15
0.09533678103755061
 Cross val 1 of 1, Repeat 7 of 15
0.09246225999239305
 Cross val 1 of 1, Repeat 8 of 15
0.09189609714113432
 Cross val 1 of 1, Repeat 9 of 15
0.09358691419919225
 Cross val 1 of 1, Repeat 10 of 15
0.09483337190017362
 Cross val 1 of 1, Repeat 11 of 15
0.088992592486007
 Cross val 1 of 1, Repeat 12 of 15
0.08215144345370738
 Cross val 1 of 1, Repeat 13 of 15
0.08076210610219126
 Cross val 1 of 1, Repeat 14 of 15
0.08469917586394654
 Cross val 1 of 1, Repeat 15 of 15
0.09383446609125458
Best Score 0.07939346495868882 at Cross val 1 of 1 and Repeat 5 of 15
The pathname is: ./xps/xpLR0/0/0/0/0//0_4_Mod.pt
 Cross val 1 of 1, Repeat 1 of 15
0.06380474435211071
 Cross val 1 

In [17]:
reps_bs_all, reps_bs_r2r_all = [], []
real_bs_all, real_bs_r2r_all = [], []

xpLR_rewrite = False

for syr in np.arange(synth_repeats):
     for inobs, nobs in enumerate(num_obs): 
        for inx,nx in enumerate(noise_lvls):

            
            pn_t1 = se.NewFolder(pn_xpLR + str(syr))
            pn_t2 = se.NewFolder(pn_t1 + str(inobs))
            pn_t3 = se.NewFolder(pn_t2 + str(inx))

            #-------------------------------------


            xbe_args = {'combs': xpLR_combs, 'pathname': pn_t3, 
                        'loggos': [False],                                                      #!!!!!!!!!!!!!!!!!!!!!!
                        'pred_scalefactors': {'nolog': scalefacts[syr][inobs][inx]['d_mode'], 
                                              'log': scalefacts[syr][inobs][inx]['d_mode_log']},
                        'split': synth_splits[syr][inobs][inx], 
                        **xbe_args_base}

            #-------------------------------------

            DY, DS = [synth_data[syr][inobs][inx][ss]
                    for ss in ['d_mode', 'd_exp']]
            
            reps_bs, reps_bs_ids = XpsBootstrapEnsemb(out = DY, out_std = DS, 
                                                      pathname_ext= 'reps',
                                                      rewrite= xpLR_rewrite,
                                                    **xbe_args)
        
            reps_bs_all.append(reps_bs) #Just get scores

            #-------------------------------------

            reps_comp = reps_bs[ref_ic]

            reps_bs_r2r = [se.PairwiseFuncer(reps_bs[ic], reps_comp, **pf_args) for ic, c in xpLR_combs]

            reps_bs_r2r_all.append(reps_bs_r2r)

            #-------------------------------------
            
            print(f'finished: {syr, inobs, inx}')


reps_bs_all, reps_bs_r2r_all = (np.stack(x) 
                                for x in [reps_bs_all, reps_bs_r2r_all])

[se.PickleDump(x, pn_xpLR + y + '_' + str(ref_ic)) 
 for x,y in zip([reps_bs_all, reps_bs_r2r_all, 
                 real_bs_all, real_bs_r2r_all],
                 ['reps_bs_all', 'reps_bs_r2r_all', 
                  'real_bs_all', 'real_bs_r2r_all'])]

False
False
False
False
False
finished: (np.int64(0), 0, 0)
False
False
False
False
False
finished: (np.int64(0), 0, 1)
False
False
False
False
False
finished: (np.int64(0), 0, 2)
False
False
False
False
False
finished: (np.int64(0), 1, 0)
False
False
False
False
False
finished: (np.int64(0), 1, 1)
False
False
False
False
False
finished: (np.int64(0), 1, 2)
False
False
False
False
False
finished: (np.int64(0), 2, 0)
False
False
False
False
False
finished: (np.int64(0), 2, 1)
False
False
False
False
False
finished: (np.int64(0), 2, 2)


[None, None, None, None]

In [20]:
xp_combs = xpLR_combs
file_ext = 'reps_bs_r2r_all'
onesided = 'lesser'
conf_alpha = 0.95

bootos = se.PickleLoad(pn_xpLR + file_ext + '_' + str(ref_ic))

syrz, prpz, nlz = synth_repeats, len(num_obs), len(noise_lvls),

ber2rs = bootos.reshape(syrz, prpz, nlz,
                            *bootos.shape[-3:])

print(ber2rs.shape)

##############################################################

ber2rs = ber2rs[:, :, :, :, :, -1] 

ber2rs_rs = np.moveaxis(ber2rs, 0, 3)

ber2rs_rs3 = ber2rs_rs.reshape(*ber2rs_rs.shape[:3], -1)

print([x.shape for x in [ber2rs, ber2rs_rs, ber2rs_rs3]])

##############################################################

(1, 3, 3, 5, 2500, 3)
[(1, 3, 3, 5, 2500), (3, 3, 5, 1, 2500), (3, 3, 5, 2500)]


In [21]:
ber2rs_mean = np.mean(ber2rs_rs3, axis = -1)
ber2rs_se = np.std(ber2rs_rs3, axis = -1)
ber2rs_low, ber2rs_high = co.BootstrapConfidenceInterval(ber2rs_rs3, alpha = conf_alpha, onesided=onesided, axis = -1)

def sigo(low, high): 
    if onesided == 'lesser': sigi = 0 > high
    if onesided == 'greater': sigi = low > 0 
    if onesided == None: sigi = np.logical_or(0 > high, low > 0)
    return sigi

ber2rs_sigo = sigo(ber2rs_low, ber2rs_high)

##################################################

per = pd.DataFrame.from_dict({(num_obs_ids[inobs], noise_lvls_ids[inx]): ber2rs_mean[inobs][inx]
                              for inobs, nobs in enumerate(num_obs)
                              for inx, nx in enumerate(noise_lvls)}).T

per.index.names = ['Num. Obs.', 'Noise Lvl']

##############################################################

if len(xp_combs[0][1]) > 1: 
    multcolx = pd.MultiIndex.from_tuples([tuple(g[1]) for g in xp_combs])
else: multcolx = [g[1][0] for g in xp_combs]
per.columns = multcolx

per_style = per.style.format(precision=4).background_gradient(axis = 1, vmin = -100, vmax = 0, cmap = 'Greys_r')


per_style = per_style.apply(lambda _: 
                            np.where(ber2rs_sigo.reshape(-1, len(multcolx)), 
                                        'font-weight: bold', ''),
                                        axis = None)

per_style

In [22]:
per.mean(axis = 0)

0.0001     0.078048
0.0007   -13.921904
0.0030   -15.090448
0.0124   -15.400500
0.0500   -14.699957
dtype: float64

# xpF: Loss Function Testing

In [10]:
optlr = 0.0124

synth_repeats = 5

In [11]:

pn_xpF = se.NewFolder(xpsfolder + 'xpF' + str(8))  #################################
#T2 corrects for adding pseudo value in deviation loss instead of in the values. 

xpF_variables = ['Loss Functions']

lossos = ['MSE', 'RMSE', 'MSZE',
          'MSLE-1', 'MSLE-0.001', 'P-NLL', 
          'ms-NMP', 'ms-NLL', 'RMSDE']

xpF_combs = [[ic, [c]] for ic, c in enumerate(lossos)]
xpF_combs

[[0, ['MSE']],
 [1, ['RMSE']],
 [2, ['MSZE']],
 [3, ['MSLE-1']],
 [4, ['MSLE-0.001']],
 [5, ['P-NLL']],
 [6, ['ms-NMP']],
 [7, ['ms-NLL']],
 [8, ['RMSDE']]]

In [ ]:
xpF_rewrite = False

for syr in np.arange(synth_repeats):
    for inobs, nobs in enumerate(num_obs):
        for inx, nx in enumerate(noise_lvls): 
            for icom, com in xpF_combs:

                pn_t1 = se.NewFolder(pn_xpF + str(syr))
                pn_t2 = se.NewFolder(pn_t1 + str(inobs))
                pn_t3 = se.NewFolder(pn_t2 + str(inx))
                pn_t4 = se.NewFolder(pn_t3 + str(icom))

                #------------------------------------------

                lsf = com[0]

                DX, DY = [synth_data[syr][inobs][inx][ss] for ss in ['d_x', 'd_mode']]

                DZ = synth_data[syr][inobs][inx]['d_exp'] if lsf in ['RMSDE', 'ms-NLL', 'ms-NMP'] else None

                #------------------------------------------

                if lsf in ['MSE', 'RMSE']: 
                    rooto = False if lsf == 'MSE' else True
                    argos = {'expo': 2, 'root': rooto}
                    lmo, mmo = [mex.AError, argos], [mex.AError, argos]
                
                if lsf in ['MSLE-1', 'MSLE-0.001']: 
                    psu = 1 if lsf == 'MSLE-1' else 0.001
                    shard = {'expo': 2, 'root': False, 'eps': psu}
                    lmo, mmo = [mex.Log_AError, {'pyt': True, **shard}], [mex.og_AError, {'pyt': False, **shard}],
                
                if lsf == 'Corr': 
                    lmo, mmo = [mex.PearsonError, {'pyt': True}], [mex.PearsonError, {'pyt': False}]
                
                if lsf == 'MSZE': 
                    meano, stdo = np.mean(DY), np.std(DY)
                    argos = {'meano': meano, 'stdo': stdo, 
                             'expo': 2, 'root': rooto}
                    lmo, mmo = [mex.Z_AError, argos], [mex.Z_AError, argos]
                
                if lsf == 'P-NLL': 
                    lmo, mmo = [mex.Poisson_NLL, {'pyt': True}], [mex.Poisson_NLL, {'pyt': False}]

                if lsf == 'ms-NMP': 
                    lmo, mmo = [mex.BetaPrime_msNMP, {'pyt': True}], [mex.BetaPrime_msNMP, {'pyt': False}]

                if lsf == 'ms-NLL': 
                    lmo, mmo = [mex.BetaPrime_msNLL, {'pyt': True}], [mex.BetaPrime_msNLL, {'pyt': False}]
                
                if lsf == 'RMSDE': 
                    shard = {'expo': 2, 'root': False, 'pseudo': 1e-10}
                    lmo, mmo = [mex.DeviaError, {'pyt': True, **shard}], [mex.DeviaError, {'pyt': False, **shard}]

                #------------------------------------------

                xpF_TT_args = {'inps': [DX], 'out': DY, 
                            'out_std': DZ, 'out_bind': None,
                            'Split': synth_splits[syr][inobs][inx],
                            'loss_mode': lmo, 'metrics_mode': mmo, 
                            **TT_base}
                
                xpF_TCS_args = {'trainer_args': xpF_TT_args, **TCS_base}

                xpF_TCR_args = {**xpF_TCS_args, **TCR_base, 'pathname': pn_t4}

                #------------------------------------------

                bo = None
                actf = tu.PseudoReLU(pseudo = 0)                                    #!!!!!!!!!!!!!!!!!!!!!!!!

                mod_args = {'learningrate': optlr,                                  #!!!!!!!!!!!!!!!!!!!!!!!!
                            'batchsize': 512, 
                            'activation_f': actf, 'two_outs': False, 
                            'comb': bo, **snx2}

                #------------------------------------------

                _, _= tu.TridentCanRepeater(pa.SimpNet, mod_args, data = None, **xpF_TCR_args) 

 Cross val 1 of 1, Repeat 1 of 15
7.5856037541155015
 Cross val 1 of 1, Repeat 2 of 15
7.778006884600296
 Cross val 1 of 1, Repeat 3 of 15
8.410250889523816
 Cross val 1 of 1, Repeat 4 of 15
16.44230562423024
 Cross val 1 of 1, Repeat 5 of 15
8.696072927424687
 Cross val 1 of 1, Repeat 6 of 15
8.788312886454143
 Cross val 1 of 1, Repeat 7 of 15
9.690111043764434
 Cross val 1 of 1, Repeat 8 of 15
8.183315511607903
 Cross val 1 of 1, Repeat 9 of 15
8.500612377951228
 Cross val 1 of 1, Repeat 10 of 15
8.539926474651656
 Cross val 1 of 1, Repeat 11 of 15
8.559621407760833
 Cross val 1 of 1, Repeat 12 of 15
8.085573955725366
 Cross val 1 of 1, Repeat 13 of 15
8.147091904945148
 Cross val 1 of 1, Repeat 14 of 15
7.987287727430961
 Cross val 1 of 1, Repeat 15 of 15
8.719564402920772
Best Score 7.5856037541155015 at Cross val 1 of 1 and Repeat 1 of 15
The pathname is: ./xps/xpF8/0/0/0/0//0_0_Mod.pt
 Cross val 1 of 1, Repeat 1 of 15
2.741129417344221
 Cross val 1 of 1, Repeat 2 of 15
2.93020128

In [43]:
rewrite_ensemb = False                                                  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

loggyz = [False] * len(xpF_combs)                                                    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

reps_bs_all, reps_bs_r2r_all = [], []
real_bs_all, real_bs_r2r_all = [], []

for syr in np.arange(synth_repeats):
     for inobs, nobs in enumerate(num_obs): 
        for inx,nx in enumerate(noise_lvls):

            
            pn_t1 = se.NewFolder(pn_xpF + str(syr))
            pn_t2 = se.NewFolder(pn_t1 + str(inobs))
            pn_t3 = se.NewFolder(pn_t2 + str(inx))

            #------------------------------------------


            xbe_args = {'combs': xpF_combs, 'pathname': pn_t3, 
                        'loggos': loggyz, 
                        'pred_scalefactors': {'nolog': 1, 
                                              'log': 1},
                        'split': synth_splits[syr][inobs][inx], 
                        **xbe_args_base, 'rewrite': rewrite_ensemb}

            #------------------------------------------

            DY, DS = [synth_data[syr][inobs][inx][ss]
                    for ss in ['d_mode', 'd_exp']]
            
            reps_bs, reps_bs_ids = XpsBootstrapEnsemb(out = DY, out_std = DS, 
                                                      pathname_ext= 'reps',
                                                    **xbe_args)
        
            reps_bs_all.append(reps_bs) #Just get scores

            #------------------------------------------

            reps_comp = reps_bs[ref_ic]

            reps_bs_r2r = [se.PairwiseFuncer(reps_bs[ic], reps_comp, **pf_args) for ic, c in xpF_combs]

            reps_bs_r2r_all.append(reps_bs_r2r)
            
            print(f'finished: {syr, inobs, inx}')


reps_bs_all, reps_bs_r2r_all = (np.array(x) for x in [reps_bs_all, reps_bs_r2r_all])

[se.PickleDump(x, pn_xpF + y + '_' + str(ref_ic)) 
 for x,y in zip([reps_bs_all, reps_bs_r2r_all],
                 ['reps_bs_all', 'reps_bs_r2r_all'])]

False
False
False
False
False
False
False
False
False
finished: (np.int64(0), 0, 0)
False
False
False
False
False
False
False
False
False
finished: (np.int64(0), 0, 1)
False
False
False
False
False
False
False
False
False
finished: (np.int64(0), 0, 2)
False
False
False
False
False
False
False
False
False
finished: (np.int64(0), 1, 0)
False
False
False
False
False
False
False
False
False
finished: (np.int64(0), 1, 1)
False
False
False
False
False
False
False
False
False
finished: (np.int64(0), 1, 2)
False
False
False
False
False
False
False
False
False
finished: (np.int64(0), 2, 0)
False
False
False
False
False
False
False
False
False
finished: (np.int64(0), 2, 1)
False
False
False
False
False
False
False
False
False
finished: (np.int64(0), 2, 2)
False
False
False
False
False
False
False
False
False
finished: (np.int64(1), 0, 0)
False
False
False
False
False
False
False
False
False
finished: (np.int64(1), 0, 1)
False
False
False
False
False
False
False
False
False
finished: (np.int64(1)

[None, None]

In [12]:
xp_combs = xpF_combs
file_ext = 'reps_bs_r2r_all'

bootos = se.PickleLoad(pn_xpF + file_ext + '_' + str(ref_ic))

syrz, prpz, nlz = synth_repeats, len(num_obs), len(noise_lvls),

ber2rs = bootos.reshape(syrz, prpz, nlz,
                            *bootos.shape[-3:])

print(ber2rs.shape)

##############################################################

ber2rs = ber2rs[:, :, :, :, :, -1]  ######################## 

ber2rs_rs = np.moveaxis(ber2rs, 0, 3)

ber2rs_rs3 = ber2rs_rs.reshape(*ber2rs_rs.shape[:3], -1)

print([x.shape for x in [ber2rs, ber2rs_rs, ber2rs_rs3]])

##############################################################

(5, 3, 3, 9, 2500, 3)
[(5, 3, 3, 9, 2500), (3, 3, 9, 5, 2500), (3, 3, 9, 12500)]


In [14]:
ber2rs_mean = np.mean(ber2rs_rs3, axis = -1)
ber2rs_se = np.std(ber2rs_rs3, axis = -1)
ber2rs_low, ber2rs_high = co.BootstrapConfidenceInterval(ber2rs_rs3, alpha = conf_alpha, onesided=onesided, axis = -1)

def sigo(low, high): 
    if onesided == 'lesser': sigi = 0 > high
    if onesided == 'greater': sigi = low > 0 
    if onesided == None: sigi = np.logical_or(0 > high, low > 0)
    return sigi

ber2rs_sigo = sigo(ber2rs_low, ber2rs_high)

##################################################

per = pd.DataFrame.from_dict({(num_obs_ids[inobs], noise_lvls_ids[inx]): ber2rs_mean[inobs][inx]
                              for inobs, nobs in enumerate(num_obs)
                              for inx, nx in enumerate(noise_lvls)})

per.index = [x[1][0] for x in xpF_combs]
per.index.names = xpF_variables

per

# ##############################################################

if len(xp_combs[0][1]) > 1: 
    multcolx = pd.MultiIndex.from_tuples([tuple(g[1]) for g in xp_combs])
else: multcolx = [g[1][0] for g in xp_combs]
# per.columns = multcolx

per_style = per.style.format(precision=1) #.background_gradient(axis = 1, vmin = -10, vmax = 0, cmap = 'Greys_r')


per_style = per_style.apply(lambda _: 
                            np.where(ber2rs_sigo.reshape(-1, len(multcolx)).T, 
                                        'font-weight: bold; background: red', ''),
                                        axis = None)

per_style

In [29]:
ber2rs_stats = np.stack([ber2rs_mean, ber2rs_se, ber2rs_low, ber2rs_high, ber2rs_sigo], axis = -1)

ber2rs_stats_tidy = [[num_obs_ids[inobs], noise_lvls_ids[inx], *c, *ber2rs_stats[inobs, inx, ic]]
                    for inobs, nobs in enumerate(num_obs)
                    for inx, nx in enumerate(noise_lvls)
                    for ic, c in xp_combs]

tups_data = [('Data', 'Num. Obs. ID', ''), 
            ('Data', 'Noise ID', '')]

tups_var = [*[('Unit', 'Subject',x) for x in xpSC_variables]]

tups_stand = [('Stat', 'Mean', ''), ('Stat', 'SE', ''),
                ('Stat', 'Percentile', '5th'), ('Stat', 'Percentile', '95th'),
                ('Stat', 'Percentile', 'Sig.')]

tups_all = [*tups_data, *tups_var, *tups_stand]

pdx = pd.DataFrame(ber2rs_stats_tidy)
multcol = pd.MultiIndex.from_tuples(tups_all)
pdx.columns = multcol

pdx[('Stat', 'Percentile', 'Sig.')] = pdx[('Stat', 'Percentile', 'Sig.')].astype('bool')

def boldo(x): 
        if x[('Stat', 'Percentile', 'Sig.')] == True: 
            return ['font-weight: bold'] * len(x) 
        else: return [''] * len(x)

pdx = pdx.sort_values([('Data', 'Num. Obs. ID', ''), ('Data', 'Noise ID', '')])

pdx_style = pdx.style.format(precision=4).apply(boldo, axis = 1).background_gradient(axis = 1, 
                                                                                        vmin = -25, vmax = 0, 
                                                                                        subset = [('Stat', 'Mean', '')], 
                                                                                        cmap = 'Greys_r')


In [30]:
pdx_style